In [1]:
from protera_stability import EmbeddingExtractor1D

In [2]:
from protera_stability import open_train_test 
from pathlib import Path

data_path = Path("../data")

> * Los `.csv` deben contener las columnas `labels` y `sequences`.

> * `data_path` debe ser Path y un directorio que contenga tus `.csv`

In [3]:
sets = open_train_test(data_path, "stability")
sets["train"]

,labels,sequence
0,0.37,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT
1,0.62,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK
2,-0.03,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK
3,1.41,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT
4,1.11,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT
...,...,...
7705,0.80,GSSKTQYEYDTKEEHQKAYEKFKKQGIPVTITQKNGKWFVQVE
7706,0.82,TIDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKPY
7707,0.66,TQDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKEY
7708,1.05,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVWVTR


In [4]:
args_dict = {
    "model_name": "esm1b_t33_650M_UR50S",
    "base_path": data_path,
    "gpu": True,
}

emb_stabilty = EmbeddingExtractor1D(**args_dict)

Using cache found in /home/roberto/.cache/torch/hub/facebookresearch_esm_master


In [5]:
embeddings = emb_stabilty.generate_embeddings(
    ["stability.csv"],  # esta en data_path
    path_out="stability_embeddings",  # guardarlo en data_path/stability_train.pkl
    bs=256,
    overwrite=True,
    target_name="stability"
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [01:07<00:00,  1.65s/it]

Embeddings saved to ../data/stability_embeddings.pkl.pkl


In [5]:
emb_stabilty.generate_datasets?

In [6]:
dset = emb_stabilty.generate_datasets(
    ["stability.csv"],
    h5_stem="stability",  # data_path / "stability".h5
    bs=256,
    embedding_file="stability_embeddings",  # cargar los embeddings desde donde se espera que esten
    target_name="stability_scores"
)

Returning existing dataset...


In [7]:
dset["labels"][:10], dset["sequences"][:10]

(array([ 0.37,  0.62, -0.03,  1.41,  1.11,  1.24,  1.05,  0.89,  0.88,
         1.15]),
 array([b'GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT',
        b'TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK',
        b'TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK',
        b'GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT',
        b'SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG',
        b'TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY',
        b'FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT',
        b'GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS'], dtype=object))

In [8]:
dset.close() # hay que cerrarlo pq es un h5py